In [2]:
import os
import numpy as np
import cv2
import imutils
import random
from math import log10, floor
from matplotlib import pyplot as plt

In [3]:
def crop(img):
    h,w = img.shape[:2]
    imgs=[]
    for i in range(0,w-224,20):
        for j in range(0, h-224,20):
            imgs.append(img[j:j+224, i:i+224])
    return imgs

def fill(img, h, w):
    img = cv2.resize(img, (w, h), cv2.INTER_CUBIC)
    return img
        
def horizontal_shift(img, ratio=0.0):
    if ratio > 1 or ratio < -1:
        print('Value should be less than 1 and greater than 0')
        return img
    h, w = img.shape
    to_shift = w*ratio
    if ratio > 0:
        img = img[:, :int(w-to_shift)]
    if ratio < 0:
        img = img[:, int(-1*to_shift):]  
    img = fill(img, h, w)
    return img

def vertical_shift(img, ratio=0.0):
    if ratio > 1 or ratio < -1:
        print('Value should be less than 1 and greater than 0')
        return img
    h, w = img.shape
    to_shift = h*ratio
    if ratio > 0:
        img = img[:int(h-to_shift), :]
    if ratio < 0:
        img = img[int(-1*to_shift):, :]
    img = fill(img, h, w)
    return img

def zoom(img, value):
    if value > 1 or value < 0:
        print('Value for zoom should be less than 1 and greater than 0')
        return img
    h, w = img.shape
    h_taken = int(value*h)
    w_taken = int(value*w)
    h_start = random.randint(0, h-h_taken)
    w_start = random.randint(0, w-w_taken)
    img = img[h_start:h_start+h_taken, w_start:w_start+w_taken]
    img = fill(img, h, w)
    return img

def horizontal_flip(img):
    return cv2.flip(img, 1)

def vertical_flip(img):
    return cv2.flip(img, 0)

def rotation(img, angle):
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h))
    return img

def highpass(img, sigma):
    return img - cv2.GaussianBlur(img, (0,0), sigma) + 127

def hist_sliding(img):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    return clahe.apply(img)

## Preprocessing
### Crop pics into 224x224

In [18]:
for i in range(1,7):
    dir_list = os.listdir(f"Dataset2/{i}")
    count=0
    for img_file in dir_list:
        img = cv2.imread(f"Dataset2/{i}/{img_file}")
        imgs = crop(img)
        
        for k in range(len(imgs)):
            cv2.imwrite(f"Dataset2/{i}/{count}.jpg", imgs[k])
            count+=1

### Augmentation

In [19]:
### Horizontal Flip
for i in range(1,7):
    dir_list = os.listdir(f"Dataset2/{i}")
    n=len(dir_list)
    print(i,end=", ")
    for img_file in dir_list:
        img = cv2.imread(f"Dataset2/{i}/{img_file}")
        img = horizontal_flip(img)
        cv2.imwrite(f"Dataset2/{i}/{n+int(img_file.split('.')[0])}.jpg", img)
print()   
### Vertical Flip
for i in range(1,7):
    dir_list = os.listdir(f"Dataset2/{i}")
    n=len(dir_list)
    print(i, end=", ")
    for img_file in dir_list:
        img = cv2.imread(f"Dataset2/{i}/{img_file}")
        img = horizontal_flip(img)
        cv2.imwrite(f"Dataset2/{i}/{n+int(img_file.split('.')[0])}.jpg", img)
print()
### Rotation by 90 degree
for i in range(1,7):
    dir_list = os.listdir(f"Dataset2/{i}")
    n=len(dir_list)
    print(i, end=", ")
    for img_file in dir_list:
        img = cv2.imread(f"Dataset2/{i}/{img_file}")
        img = horizontal_flip(img)
        cv2.imwrite(f"Dataset2/{i}/{n+int(img_file.split('.')[0])}.jpg", img)

1, 2, 3, 4, 5, 6, 
1, 2, 3, 4, 5, 6, 
1, 2, 3, 4, 5, 6, 

## GLCM

In [4]:
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import shannon_entropy

X = []; y = []
for i in range(1,7):
    dir_list = os.listdir(f"Dataset2/{i}")
    n = len(dir_list)
    print(i)
    for j in range(5000):
        img_file = dir_list[j]
        distances = [1, 2, 3]
        angles = [0, np.pi/6, np.pi/3, np.pi/2]
        properties = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
        y.append(i)
        img = cv2.imread(f"Dataset2/{i}/{img_file}")
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        glcm = graycomatrix(img, distances=distances, angles=angles, symmetric=True, normed=True)

        feats = np.hstack([graycoprops(glcm, prop).ravel() for prop in properties])
        entropy = [shannon_entropy(glcm[:,:,a,b]) for a in range(len(distances)) for b in range(len(angles))]
        feats = np.hstack((feats, entropy))

        X.append(feats)

X = np.array(X)
y = np.array(y)         
print(X.shape, y.shape)


1
2
3
4
5
6
(30000, 84) (30000,)


In [5]:
np.savetxt("Dataset2/dataset_x.csv", X, delimiter=",")
np.savetxt("Dataset2/dataset_y.csv", y, delimiter=",")
np.savetxt("Dataset2/dataset.csv", np.hstack((X,y.reshape(30000,1))), delimiter=",")

## Models

In [6]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

dataset = pd.read_csv('Dataset2/dataset.csv', header=None)
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1:].values

print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

(30000, 84) (30000, 1)
(24000, 84) (6000, 84) (24000, 1) (6000, 1)


### SVM

In [7]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_train, y_train.ravel())
joblib.dump(clf, "Models2/svm.joblib")
model = joblib.load("Models2/svm.joblib")
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred, target_names=["1","2","3","4","5","6"], digits=5))
print(confusion_matrix(y_test, y_pred))
print("ACCURACY OF THE MODEL: ", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           1    0.95077   0.92700   0.93873      1000
           2    1.00000   1.00000   1.00000      1000
           3    0.97460   0.92100   0.94704      1000
           4    0.92122   0.99400   0.95623      1000
           5    0.99794   0.96800   0.98274      1000
           6    0.93598   0.96500   0.95027      1000

    accuracy                        0.96250      6000
   macro avg    0.96342   0.96250   0.96250      6000
weighted avg    0.96342   0.96250   0.96250      6000

[[ 927    0    9    0    1   63]
 [   0 1000    0    0    0    0]
 [  13    0  921   66    0    0]
 [   0    0    5  994    1    0]
 [   4    0    7   18  968    3]
 [  31    0    3    1    0  965]]
ACCURACY OF THE MODEL:  0.9625


### Decision Tree J48

In [8]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train.ravel())
joblib.dump(clf, "Models2/j48.joblib")
model = joblib.load("Models2/j48.joblib")
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred, target_names=["1","2","3","4","5","6"], digits=5))
print(confusion_matrix(y_test, y_pred))
print("ACCURACY OF THE MODEL: ", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           1    0.97140   0.98500   0.97815      1000
           2    0.99800   0.99800   0.99800      1000
           3    0.96703   0.96800   0.96752      1000
           4    0.96439   0.97500   0.96967      1000
           5    0.98468   0.96400   0.97423      1000
           6    0.98392   0.97900   0.98145      1000

    accuracy                        0.97817      6000
   macro avg    0.97824   0.97817   0.97817      6000
weighted avg    0.97824   0.97817   0.97817      6000

[[985   0   4   0   2   9]
 [  0 998   0   1   1   0]
 [  7   0 968  23   1   1]
 [  0   0  17 975   8   0]
 [  7   2  11  10 964   6]
 [ 15   0   1   2   3 979]]
ACCURACY OF THE MODEL:  0.9781666666666666


### Random Forest

In [9]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators = 100) 
clf.fit(X_train, y_train.ravel())
joblib.dump(clf, "Models2/rf.joblib")
model = joblib.load("Models2/rf.joblib")
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred, target_names=["1","2","3","4","5","6"], digits=5))
print(confusion_matrix(y_test, y_pred))
print("ACCURACY OF THE MODEL: ", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           1    0.99499   0.99400   0.99450      1000
           2    1.00000   1.00000   1.00000      1000
           3    0.99696   0.98500   0.99095      1000
           4    0.98225   0.99600   0.98908      1000
           5    0.99799   0.99100   0.99448      1000
           6    0.99304   0.99900   0.99601      1000

    accuracy                        0.99417      6000
   macro avg    0.99421   0.99417   0.99417      6000
weighted avg    0.99421   0.99417   0.99417      6000

[[ 994    0    0    0    0    6]
 [   0 1000    0    0    0    0]
 [   4    0  985   11    0    0]
 [   0    0    2  996    2    0]
 [   0    0    1    7  991    1]
 [   1    0    0    0    0  999]]
ACCURACY OF THE MODEL:  0.9941666666666666


### ANN

In [12]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

y_train_new = []
for y in y_train:
    if y==1:
        y_train_new.append([1,0,0,0,0,0])
    if y==2:
        y_train_new.append([0,1,0,0,0,0])
    if y==3:
        y_train_new.append([0,0,1,0,0,0])
    if y==4:
        y_train_new.append([0,0,0,1,0,0])
    if y==5:
        y_train_new.append([0,0,0,0,1,0])
    if y==6:
        y_train_new.append([0,0,0,0,0,1])
y_train = np.array(y_train_new)

ann = tf.keras.models.Sequential()
# Adding layers
ann.add(tf.keras.layers.Dense(units=256,activation="relu"))
ann.add(tf.keras.layers.Dense(units=64,activation="relu"))
ann.add(tf.keras.layers.Dense(units=6,activation="sigmoid"))
ann.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

checkpoint = ModelCheckpoint('Models2/glcm_ann_{epoch:01d}.h5', save_freq=5*750)
ann.fit(X_train,y_train,batch_size=32,epochs = 50, callbacks=[checkpoint] )

Epoch 1/50
750/750 [==============================] - 4s 5ms/step - loss: 0.1041 - accuracy: 0.8711
Epoch 2/50
750/750 [==============================] - 2s 3ms/step - loss: 0.0434 - accuracy: 0.9507
Epoch 3/50
750/750 [==============================] - 4s 5ms/step - loss: 0.0295 - accuracy: 0.9695
Epoch 4/50
750/750 [==============================] - 3s 4ms/step - loss: 0.0218 - accuracy: 0.9775
Epoch 5/50
750/750 [==============================] - 3s 5ms/step - loss: 0.0177 - accuracy: 0.9817
Epoch 6/50
750/750 [==============================] - 3s 4ms/step - loss: 0.0161 - accuracy: 0.9851
Epoch 7/50
750/750 [==============================] - 3s 5ms/step - loss: 0.0120 - accuracy: 0.9878
Epoch 8/50
750/750 [==============================] - 3s 4ms/step - loss: 0.0117 - accuracy: 0.9880
Epoch 9/50
750/750 [==============================] - 3s 4ms/step - loss: 0.0102 - accuracy: 0.9898
Epoch 10/50
750/750 [==============================] - 3s 4ms/step - loss: 0.0102 - accuracy: 0.9894

In [30]:
for i in range(5,55,5):
    model = tf.keras.models.load_model(f"Models2/glcm_ann_{i}.h5")
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)+1
    print(y_pred.shape)
    print("#####################")
    print(f"Model{i}")
    print()
    print(classification_report(y_test, y_pred, target_names=["1","2","3","4","5","6"], digits=5))
    print(confusion_matrix(y_test, y_pred))
    print("ACCURACY OF THE MODEL: ", accuracy_score(y_test, y_pred))
    print()

(6000,)
#####################
Model5

              precision    recall  f1-score   support

           1    0.99492   0.97900   0.98690      1000
           2    1.00000   1.00000   1.00000      1000
           3    0.97892   0.97500   0.97695      1000
           4    0.97624   0.98600   0.98109      1000
           5    1.00000   0.99300   0.99649      1000
           6    0.98132   0.99800   0.98959      1000

    accuracy                        0.98850      6000
   macro avg    0.98856   0.98850   0.98850      6000
weighted avg    0.98856   0.98850   0.98850      6000

[[ 979    0    3    0    0   18]
 [   0 1000    0    0    0    0]
 [   3    0  975   22    0    0]
 [   0    0   14  986    0    0]
 [   0    0    4    2  993    1]
 [   2    0    0    0    0  998]]
ACCURACY OF THE MODEL:  0.9885

(6000,)
#####################
Model10

              precision    recall  f1-score   support

           1    0.99302   0.99600   0.99451      1000
           2    1.00000   1.00000   1.00

In [32]:
for i in range(1,7):
    img = cv2.imread(f"Dataset2/{i}/1.jpg")
    cv2.imwrite(f"Dataset2/{0}/i_1.jpg", img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    histr = cv2.calcHist([img],[0],None,[256],[0,256])
    plt.plot(histr)
    plt.savefig(f"Dataset2/0/{i}_hist_before")
    plt.close()
    cv2.imwrite(f"Dataset2/0/{i}_1_1.jpg", img)
    img = hist_sliding(img)
    histr = cv2.calcHist([img],[0],None,[256],[0,256])
    plt.plot(histr)
    plt.savefig(f"Dataset2/0/{i}_hist_after")
    plt.close()
    cv2.imwrite(f"Dataset2/0/{i}_1_2.jpg", img)
    img = highpass(img, 5)
    cv2.imwrite(f"Dataset2/0/{i}_1_3.jpg", img)
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    cv2.imwrite(f"Dataset2/0/{i}_1_4.jpg", img)

###### 